In [1]:
import requests
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
from random import randint
from time import sleep

In [2]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0"}
url = 'https://www.vivino.com/'

# Get Cache key to get country codes and type of wines
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
script = soup.find('script', text = re.compile('var vivinoCacheKey'))
vivinoCacheKey = str(script).split('vivinoCacheKey = ')[-1].split(';')[0].replace("'",'').strip()

# Get countries
api_url = 'https://www.vivino.com/api/countries'
payload = {
    'cache_key':vivinoCacheKey}
countryData = requests.get(api_url, headers=headers, params=payload).json()['countries']

rows = []
# Iterate through countries and wine types
api_url = 'https://www.vivino.com/api/explore/explore'
for country in countryData:
    payload = {
    "country_code": country['code'].upper(),
    "currency_code":country['currency']['code'],
    'grape_ids[]':'131',
    "grape_filter":"varietal",
    "min_rating":"1",
    "order_by":"ratings_count",
    "order":"desc",
    "page": '1',
    "price_range_max":"1000",
    "price_range_min":"1"}

    try:
        jsonData = requests.get(api_url, params=payload, headers=headers).json()
        total_pages = math.ceil(jsonData['explore_vintage']['records_matched'] / 100)
        #print('%s' %(country['code'].upper()))
        
        for page in range(1,total_pages+1):
            if page != 1:   
                payload.update({'page':page})
            jsonData = requests.get(api_url, params=payload, headers=headers).json()
            for each in jsonData['explore_vintage']['records']:
                name = f'{each["vintage"]["wine"]["name"]} {each["vintage"]["year"]}'
                year = each["vintage"]["year"]
                id = each["vintage"]["wine"]["id"]
                rating =  each['vintage']['statistics']['ratings_average']
                price = each['price']['amount']
                winery = each["vintage"]["wine"]["winery"]["name"]
                ratings_count = each["vintage"]["statistics"]["ratings_count"]
                country = country
                acidity = each['vintage']['wine']['taste']['structure']['acidity']
                calculated_structure_count = each['vintage']['wine']['taste']['structure']['calculated_structure_count']
                intensity = each['vintage']['wine']['taste']['structure']['intensity']
                sweetness = each['vintage']['wine']['taste']['structure']['sweetness']
                tannin = each['vintage']['wine']['taste']['structure']['tannin'],
                user_structure_count = each['vintage']['wine']['taste']['structure']['user_structure_count'],
                flavor_1 = each['vintage']['wine']['taste']['flavor'][0]['group'],
                flavor_1_count = each['vintage']['wine']['taste']['flavor'][0]['stats']['count'],
                flavor_2 = each['vintage']['wine']['taste']['flavor'][1]['group'],
                flavor_2_count = each['vintage']['wine']['taste']['flavor'][1]['stats']['count'],
                flavor_3 = each['vintage']['wine']['taste']['flavor'][2]['group'],
                flavor_3_count = each['vintage']['wine']['taste']['flavor'][2]['stats']['count'],
                flavor_4 = each['vintage']['wine']['taste']['flavor'][3]['group'],
                flavor_4_count = each['vintage']['wine']['taste']['flavor'][3]['stats']['count'],
                flavor_5 = each['vintage']['wine']['taste']['flavor'][4]['group'],
                flavor_5_count = each['vintage']['wine']['taste']['flavor'][4]['stats']['count'],

                
                
                row = {'name':name, "year":year, "wine ID":id, 'rating':rating, 'price':price, 'winery': winery, 'ratings_count': ratings_count, 'country': country,\
                       'acidity': acidity, 'calculated_structure_count': calculated_structure_count, 'intensity': intensity, 'sweetness': sweetness, 'tannin': tannin, 'user_structure_count': user_structure_count,\
                        'flavor_1': flavor_1, 'flavor_1_count': flavor_1_count, 'flavor_2': flavor_2, 'flavor_2_count': flavor_2_count, 'flavor_3': flavor_3, 'flavor_3_count': flavor_3_count, 'flavor_4': flavor_4, 'flavor_4_count': flavor_4_count, 'flavor_5': flavor_5, 'flavor_5_count': flavor_5_count}
                rows.append(row)
            print('Aquired page: %s - %s ' %(country['code'].upper(), page))
    except:
        continue

df = pd.DataFrame(rows)

Aquired page: AD - 1 


In [2]:
ratings=[]
for _, row in df.iterrows():
    page = 1
    while True:
        print(
            f'Getting info about wine {row["wine ID"]}-{row["year"]} Page {page}'
            )
        try:
            d = get_wine_data(row["wine ID"], row["year"], page)
        except:
            sleep(10)
            continue

        try:
            for r in d["reviews"]:
                if r["language"] != "en": # <-- get only english reviews
                    continue

                ratings.append(
                        [
                            row["wine ID"],
                            r["rating"],
                            r["note"],
                            r["created_at"],
                        ]
                    )
        except:
            break


        if page == 5:
            sleep(randint(2,8))
            break

        page += 1


    df_reviews = pd.DataFrame(
        ratings, columns=["wine ID", "User Rating", "Review", "CreatedAt"]
    )

    df_out = df_reviews.merge(df)
df_out.to_csv("reviews.csv", index=False)

Aquired page: AS - 1 
Aquired page: AD - 1 
Aquired page: AD - 2 
Aquired page: AD - 3 
Aquired page: AD - 4 
Aquired page: AD - 5 
Aquired page: AD - 6 
Aquired page: AD - 7 
Aquired page: AD - 8 
Aquired page: AD - 9 
Aquired page: AR - 1 
Aquired page: AM - 1 
Aquired page: AM - 2 
Aquired page: AU - 1 
Aquired page: AU - 2 
Aquired page: AU - 3 
Aquired page: AU - 4 
Aquired page: AU - 5 
Aquired page: AU - 6 
Aquired page: AU - 7 
Aquired page: AU - 8 
Aquired page: AU - 9 
Aquired page: AU - 10 
Aquired page: AU - 11 
Aquired page: AU - 12 
Aquired page: AT - 1 
Aquired page: AT - 2 
Aquired page: AT - 3 
Aquired page: AT - 4 
Aquired page: AT - 5 
Aquired page: AT - 6 
Aquired page: AT - 7 
